<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/main/THESIS_MODEL_V0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [ ]:
!pip install gekko

     |████████████████████████████████| 10.8MB 3.8MB/s 


**Importing libraries needed**

In [ ]:
import numpy as np
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")



Mounted at /content/drive


**UPLOADING FOREST INVENTORY DATA**

In [ ]:
#Upload distances from each hexagon to other
Dist = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Hardwood_Sawlog = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")


**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [67]:
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization

#define parameter
Num_Cell= 2010

H2 = m.Array(m.Var,(Num_Cell),value = 0)
Demand = 20000000 
D = np.zeros(Num_Cell)
F = Hardwood_Sawlog.SumOfTotal

for i in range(Num_Cell):
  H2[i].lower = 0
  H2[i].upper = 1000000
  D[i] = i

m.Equation(m.sum(H2[0:Num_Cell])==Demand)
m.Minimize(m.sum([D[i]*H2[i] for i in range(len(H2))])) 
m.solve(disp=True,debug=True)

print('') 
print(H2)

0       True
1       True
2       True
3       True
4       True
        ... 
7865    True
7866    True
7867    True
7868    True
7869    True
Name: SumOfTotal, Length: 7870, dtype: bool